In [1]:
import numpy as np
import pandas as pd

#!pip install urllib
import urllib.request as url 

#!pip install bs4
from bs4 import BeautifulSoup as bs

#!pip install wordcloud
from wordcloud import WordCloud

import re
import matplotlib.pyplot as plt
%matplotlib inline

import random
random.seed(123)

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!conda install -c conda-forge spacy
#!python -m spacy download en_core_web_sm
#!pip install -U spacy
import spacy
nlp = spacy.load("en_core_web_sm")

# import en_core_web_sm
#nlp = en_core_web_sm.load()

# Download 100 text books

In [4]:
import requests
from bs4 import BeautifulSoup
def go_gutenberg(file_format,book_no):
       url = "https://www.gutenberg.org/ebooks/" + str(book_no)
       r =requests.get(url)
       r_html  = r.text
       soup = BeautifulSoup(r_html,"html.parser")
       for file in soup.find_all('a',class_="link"):
         if file_format in file.text:
           get_book=file.get('href')
           g = requests.get("https://www.gutenberg.org/"+get_book)

           with open("C:/Users/91989/Documents/PYTHON/INSOFE_Internship/Task3/Book_author"+str(book_no)+".txt",'wb') as open_file:
               for chunk in g.iter_content(10000):
                    open_file.write(chunk)


In [5]:
book_num = range(1,101)
for x in book_num:
    go_gutenberg("Text",x)

KeyboardInterrupt: 

# get author names for those 100 books

In [6]:
d1={'author':[],'book_num':[]}
def extract_author(book_num):
    url = "https://www.gutenberg.org/ebooks/" + str(book_num)
    r =requests.get(url)
    r_html  = r.text
    soup = BeautifulSoup(r_html,"html.parser")
    #d1={'author':[]}
    for link in soup.find_all('a'):
         if len(link.text.strip()) > 1 and bool(
            re.search('ebooks/author',link['href'])) and not bool(
            re.search('by',link.text)):
            d1['author'].append(link.text)
            d1['book_num'].append(book_num)
    #blog_list2 = pd.DataFrame(d1)       
    #return blog_list2
#extract_author(1)

In [7]:
book_num = range(1,101)
for x in book_num:
    extract_author(x)

In [8]:
author_names=pd.DataFrame(d1)

In [9]:
author_names.head()

,author,book_num
0,"Jefferson, Thomas, 1743-1826",1
1,United States,2
2,"Kennedy, John F. (John Fitzgerald), 1917-1963",3
3,"Lincoln, Abraham, 1809-1865",4
4,United States,5


In [10]:
author_names.shape

(109, 2)

In [11]:
author_names.head()

,author,book_num
0,"Jefferson, Thomas, 1743-1826",1
1,United States,2
2,"Kennedy, John F. (John Fitzgerald), 1917-1963",3
3,"Lincoln, Abraham, 1809-1865",4
4,United States,5


# read all books and book number in dataframe

In [12]:
import os
path='C:/Users/91989/Documents/PYTHON/INSOFE_Internship/Task3/Book_author/data'
all_files = os.listdir(path)
new_list = {'text':[],'name':[]}
for fle in all_files:
   # open the file and then call .read() to get the text
   with open(os.path.join(path, fle),"rb") as f:
      text = f.read()
      new_list['text'].append(text)
      new_list['name'].append(fle)

In [13]:
train_data=pd.DataFrame(new_list)

In [14]:
train_data.head()

,text,name
0,b'\xef\xbb\xbf\r\n\r\n========================...,Books1.txt
1,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,Books10.txt
2,b'\xef\xbb\xbf\r\nProject Gutenberg\xe2\x80\x9...,Books100.txt
3,b'The Project Gutenberg EBook of Alice\xe2\x80...,Books11.txt
4,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,Books12.txt


In [15]:
train_data['name']=train_data.name.apply(lambda name: name.split('.')[0].strip())

In [16]:
train_data['name']=train_data.name.apply(lambda name: name.split('s')[1].strip())

In [17]:
train_data.head()

,text,name
0,b'\xef\xbb\xbf\r\n\r\n========================...,1
1,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,10
2,b'\xef\xbb\xbf\r\nProject Gutenberg\xe2\x80\x9...,100
3,b'The Project Gutenberg EBook of Alice\xe2\x80...,11
4,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,12


In [18]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
text    99 non-null object
name    99 non-null object
dtypes: object(2)
memory usage: 1.6+ KB


In [19]:
train_data['name'] = train_data['name'].astype('int')

In [20]:
train_data=train_data.sort_values(by=['name'])

In [21]:
train_data.head()

,text,name
0,b'\xef\xbb\xbf\r\n\r\n========================...,1
12,b'\xef\xbb\xbf**Welcome To The World of Free P...,2
23,b'\xef\xbb\xbf**Welcome To The World of Free P...,3
34,b'\xef\xbb\xbf\r\n\r\n**Welcome To The World o...,4
44,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,5


In [22]:
train_data.shape

(99, 2)

In [23]:
train_data.reset_index(inplace=True)

In [24]:
train_data.head()

,index,text,name
0,0,b'\xef\xbb\xbf\r\n\r\n========================...,1
1,12,b'\xef\xbb\xbf**Welcome To The World of Free P...,2
2,23,b'\xef\xbb\xbf**Welcome To The World of Free P...,3
3,34,b'\xef\xbb\xbf\r\n\r\n**Welcome To The World o...,4
4,44,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,5


In [25]:
train_data.drop("index",axis=1,inplace=True)

In [26]:
train_data.columns=['book_text','book_num']

In [27]:
train_data.head()

,book_text,book_num
0,b'\xef\xbb\xbf\r\n\r\n========================...,1
1,b'\xef\xbb\xbf**Welcome To The World of Free P...,2
2,b'\xef\xbb\xbf**Welcome To The World of Free P...,3
3,b'\xef\xbb\xbf\r\n\r\n**Welcome To The World o...,4
4,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,5


# merge both dataframes

In [28]:
train_data_merge=pd.merge(train_data,author_names , on='book_num', how='outer')

In [29]:
train_data_merge.head()

,book_text,book_num,author
0,b'\xef\xbb\xbf\r\n\r\n========================...,1,"Jefferson, Thomas, 1743-1826"
1,b'\xef\xbb\xbf**Welcome To The World of Free P...,2,United States
2,b'\xef\xbb\xbf**Welcome To The World of Free P...,3,"Kennedy, John F. (John Fitzgerald), 1917-1963"
3,b'\xef\xbb\xbf\r\n\r\n**Welcome To The World o...,4,"Lincoln, Abraham, 1809-1865"
4,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,5,United States


In [33]:
train_data_merge.isnull().sum()

book_text    0
book_num     0
author       0
dtype: int64

In [31]:
train_data_merge.dropna(inplace=True)

In [32]:
train_data_merge.shape

(108, 3)

In [92]:
#Duplicating the original text extracted before proceeeding with preprocessing steps

import copy
data = copy.deepcopy(train_data_merge)

In [93]:
data.head()

,book_text,book_num,author
0,b'\xef\xbb\xbf\r\n\r\n========================...,1,"Jefferson, Thomas, 1743-1826"
1,b'\xef\xbb\xbf**Welcome To The World of Free P...,2,United States
2,b'\xef\xbb\xbf**Welcome To The World of Free P...,3,"Kennedy, John F. (John Fitzgerald), 1917-1963"
3,b'\xef\xbb\xbf\r\n\r\n**Welcome To The World o...,4,"Lincoln, Abraham, 1809-1865"
4,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,5,United States


In [94]:
data.drop("book_num",axis=1,inplace=True)

In [95]:
data.columns=['text','author']

In [96]:
data.head()

,text,author
0,b'\xef\xbb\xbf\r\n\r\n========================...,"Jefferson, Thomas, 1743-1826"
1,b'\xef\xbb\xbf**Welcome To The World of Free P...,United States
2,b'\xef\xbb\xbf**Welcome To The World of Free P...,"Kennedy, John F. (John Fitzgerald), 1917-1963"
3,b'\xef\xbb\xbf\r\n\r\n**Welcome To The World o...,"Lincoln, Abraham, 1809-1865"
4,b'\xef\xbb\xbfThe Project Gutenberg EBook of T...,United States


# Text Preprocessing

### LowerCase all text

In [97]:
data['text'] = [text.strip().lower() for text in data['text']]
data['text'][:10]

0     b'\xef\xbb\xbf\r\n\r\n========================...
1     b'\xef\xbb\xbf**welcome to the world of free p...
2     b'\xef\xbb\xbf**welcome to the world of free p...
3     b'\xef\xbb\xbf\r\n\r\n**welcome to the world o...
4     b'\xef\xbb\xbfthe project gutenberg ebook of t...
5     b'\xef\xbb\xbf**welcome to the world of free p...
7     b'\xef\xbb\xbfthe project gutenberg ebook of l...
8     b'\xef\xbb\xbfthe project gutenberg ebook of l...
10    b'the project gutenberg ebook of alice\xe2\x80...
11    b'\xef\xbb\xbfthe project gutenberg ebook of t...
Name: text, dtype: object

## Defining the functions to perform basic steps like 

- **expanding contractions**
 
- **remove accented characters**

- **scrub words**

In [98]:
type(data['text'])

pandas.core.series.Series

###  Handling contractions 

In [99]:
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [100]:
def expand_contractions(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    return text

In [101]:
data['text'] = data['text'].astype(str)

In [102]:
import re
data['text'] = [expand_contractions(re.sub('’', "'", text)) for text in data['text']]
data['text'][1]

'b\'\\xef\\xbb\\xbf**welcome to the world of free plain vanilla electronic texts**\\r\\n\\r\\n**etexts readable by both humans and by computers, since 1971**\\r\\n\\r\\n*these etexts prepared by hundreds of volunteers and donations*\\r\\n\\r\\n\\r\\ndecember, 1972  [etext #2]\\r\\n\\r\\n\\r\\n****the project gutenberg etext of the u. s. bill of rights****\\r\\n*******this file should be named 2.txt or 2.zip******\\r\\n\\r\\nwe apologize for the fact that the legal small print is longer,\\r\\nand more complicated, than the etext itself, our legal beagles,\\r\\nof whom there are now a half dozen or so, insist this must be a\\r\\npart of any project gutenberg etext we post, for our protection\\r\\nfrom the rest of the legal beagles out there.  the us has twice\\r\\nas many lawyers as the rest of the world combined!\\r\\n\\r\\nyou are free to delete the headers and just keep the etexts, we\\r\\nare not free not to post it this way.  again my apologies.  the\\r\\nnormal project gutenberg bl

## Invoking the remove_accented_chars() function

In [103]:
import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #https://docs.python.org/2/library/unicodedata.html
    return text

In [104]:
data['text'] = [remove_accented_chars(text) for text in data['text']]
data['text'][2]

'b\'\\xef\\xbb\\xbf**welcome to the world of free plain vanilla electronic texts**\\r\\n\\r\\n**etexts readable by both humans and by computers, since 1971**\\r\\n\\r\\n*these etexts prepared by hundreds of volunteers and donations*\\r\\n\\r\\n\\r\\n**the project gutenberg etext of kennedy\\\'s inaugural address**\\r\\n*******this file should be named 3.txt or 3.zip*******\\r\\n\\r\\n\\r\\n[etext #3] november 22, 1973, 10th anniversary of assassination\\r\\n[officially rereleased for november 22, 1993, 30th anniversary]\\r\\n\\r\\nwe apologize for the fact that the legal small print is longer,\\r\\nand more complicated, than the etext itself, our legal beagles,\\r\\nof whom there are now a half dozen or so, insist this must be a\\r\\npart of any project gutenberg etext we post, for our protection\\r\\nfrom the rest of the legal beagles out there.  the us has twice\\r\\nas many lawyers as the rest of the world combined!\\r\\n\\r\\nyou are free to delete the headers and just keep the ete

## Invoking various scrub functions

In [105]:
def scrub_words(text):
    #Replace \xao characters in text
    text = re.sub('\xa0', ' ', text)
    
    #Replace non ascii / not words and digits
    text = re.sub("(\\W|\\d)",' ',text)
    
    #Replace new line characters and following text untill space
    text = re.sub('\n(\w*?)[\s]', '', text)
    
    #Remove html markup
    text = re.sub("<.*?>", ' ', text)
    
    #Remove extra spaces from the text
    text = re.sub("\s+", ' ', text)
    return text

In [106]:
data['text'] = [scrub_words(text) for text in data['text']]
data['text'][1]

'b xef xbb xbf welcome to the world of free plain vanilla electronic texts r n r n etexts readable by both humans and by computers since r n r n these etexts prepared by hundreds of volunteers and donations r n r n r ndecember etext r n r n r n the project gutenberg etext of the u s bill of rights r n this file should be named txt or zip r n r nwe apologize for the fact that the legal small print is longer r nand more complicated than the etext itself our legal beagles r nof whom there are now a half dozen or so insist this must be a r npart of any project gutenberg etext we post for our protection r nfrom the rest of the legal beagles out there the us has twice r nas many lawyers as the rest of the world combined r n r nyou are free to delete the headers and just keep the etexts we r nare not free not to post it this way again my apologies the r nnormal project gutenberg blurb has been deleted you can get it r nin this location in most project gutenberg etexts thanks mh r n r n r n st

# Text Preprocessing

In [107]:
data.head()

,text,author
0,b xef xbb xbf r n r n r n r n note this file c...,"Jefferson, Thomas, 1743-1826"
1,b xef xbb xbf welcome to the world of free pla...,United States
2,b xef xbb xbf welcome to the world of free pla...,"Kennedy, John F. (John Fitzgerald), 1917-1963"
3,b xef xbb xbf r n r n welcome to the world of ...,"Lincoln, Abraham, 1809-1865"
4,b xef xbb xbfthe project gutenberg ebook of th...,United States


#### Converting dataframe back to dictionary

In [108]:
data = data.reset_index().to_dict(orient='list')
print(f"The keys in the dictionary are {data.keys()}")

The keys in the dictionary are dict_keys(['index', 'text', 'author'])


In [109]:
print(data['text'][5])

b xef xbb xbf welcome to the world of free plain vanilla electronic texts r n r n etexts readable by both humans and by computers since r n r n these etexts prepared by hundreds of volunteers and donations r n r n r nthe project gutenberg etext of give me liberty or give me death r n r n this file should be named txt or zip r n r ndecember etext r n date last updated may r n r nofficially released in december unofficially released for r nthe th anniversary of the speech by patrick henry before the r n house as he referred to it which was the virgina provincial r nconvention march r n r nwe apologize for the fact that the legal small print is longer r nand more complicated than the etext itself our legal beagles r nof whom there are now a half dozen or so insist this must be a r npart of any project gutenberg etext we post for our protection r nfrom the rest of the legal beagles out there the us has twice r nas many lawyers as the rest of the world combined r n r nyou are free to delete

In [110]:
type(data['text'])

list

## Stopwords, stemming, and tokenizing

In [111]:
#!conda install -c conda-forge spacy
#!python -m spacy download en_core_web_sm
#!pip install -U spacy
import spacy
nlp = spacy.load("en_core_web_sm")

# import en_core_web_sm
#nlp = en_core_web_sm.load()

In [112]:
## Adding Custom stopwords to the spacy stopword list
customize_stop_words = ['xef','xbb','xbf']

for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

In [113]:
## It might be surprising, but spaCy doesn't contain any function for stemming as it relies on lemmatization only. 
## Therefore, in this section, we will use NLTK for stemming.

## load nltk's SnowballStemmer as variable 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [114]:
# Here I define a tokenizer and stemmer which returns the set of stems (excluding stop words) in the text that it is passed

def tokenize_and_stem(doc, remove_stopwords = True):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    if remove_stopwords:
        tokens = [word.text for word in doc if not word.is_stop]
    else:
        tokens = [word.text for word in doc]

    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)

    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_and_lemmatize(doc, remove_stopwords = True):
    
    if remove_stopwords:
        tokens = [word for word in doc if not word.is_stop]
    else:
        tokens = [word for word in doc]
        
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token.text):
            filtered_tokens.append(token)
            
    lemma = [t.lemma_ for t in filtered_tokens]
    return lemma


def tokenize_only(doc, remove_stopwords = True):
    
    if remove_stopwords:
        tokens = [word.text for word in doc if not word.is_stop]
    else:
        tokens = [word.text for word in doc]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [116]:
data=pd.DataFrame(data)

In [119]:
data=data.drop([107])

In [121]:
data = data.reset_index().to_dict(orient='list')
print(f"The keys in the dictionary are {data.keys()}")

The keys in the dictionary are dict_keys(['level_0', 'index', 'text', 'author'])


In [123]:
nlp = spacy.load('en_core_web_sm') 
nlp.max_length =2500000

In [124]:
%%time
## We are trying to create four seperate lists for text with stop words, text without stop words,
## text with stemmed words and text with lemmatized words.

## Naming Conventions followed ####

## 'clean' word is appended to lists which do not contain stopwords

## 'all' keyword is appended to lists which contain stopwords.

## use extend so it's a big flat list of vocab

data['clean_text_stemmed'] = []
data['clean_text_lemmatized'] = []
data['text_stemmed'] = []
data['text_lemmatized'] = []

vocab_stemmed = []
allvocab_stemmed =[]

vocab_tokenized = []
allvocab_tokenized = []

vocab_lemmatized = []
allvocab_lemmatized = []


for idx,text in enumerate(data['text']):

    doc = nlp(text)
    print(f"processing {idx} document")
    words_stemmed = tokenize_and_stem(doc)
    vocab_stemmed.extend(words_stemmed)
    data['clean_text_stemmed'].append(words_stemmed)
        
    words_lemmatized = tokenize_and_lemmatize(doc)
    vocab_lemmatized.extend(words_lemmatized)
    data['clean_text_lemmatized'].append(words_lemmatized)
    
       
    allwords_stemmed = tokenize_and_stem(doc, False)
    allvocab_stemmed.extend(allwords_stemmed)
    data['text_stemmed'].append(allwords_stemmed)
    
    allwords_lemmatized = tokenize_and_lemmatize(doc, False)
    allvocab_lemmatized.extend(allwords_lemmatized)
    data['text_lemmatized'].append(allwords_lemmatized)
    
    allwords_tokenized = tokenize_only(doc,False)
    allvocab_tokenized.extend(allwords_tokenized)
    
    words_tokenized = tokenize_only(doc)
    vocab_tokenized.extend(words_tokenized)

processing 0 document
processing 1 document
processing 2 document
processing 3 document
processing 4 document
processing 5 document
processing 6 document
processing 7 document
processing 8 document
processing 9 document
processing 10 document
processing 11 document
processing 12 document
processing 13 document
processing 14 document
processing 15 document
processing 16 document
processing 17 document
processing 18 document
processing 19 document
processing 20 document
processing 21 document
processing 22 document
processing 23 document
processing 24 document
processing 25 document
processing 26 document
processing 27 document
processing 28 document
processing 29 document
processing 30 document
processing 31 document
processing 32 document
processing 33 document
processing 34 document
processing 35 document
processing 36 document
processing 37 document
processing 38 document
processing 39 document
processing 40 document
processing 41 document
processing 42 document
processing 43 documen

In [125]:
# save the model to disk
import pickle
filename = 'data.pkl'
pickle.dump(data, open(filename, 'wb'))

In [3]:
import pickle
data = pickle.load(open('data.pkl', 'rb'))

In [126]:
pd.DataFrame(data).head()

,level_0,index,text,author,clean_text_stemmed,clean_text_lemmatized,text_stemmed,text_lemmatized
0,0,0,b xef xbb xbf r n r n r n r n note this file c...,"Jefferson, Thomas, 1743-1826","[b, xef, xbb, xbf, r, n, r, n, r, n, r, n, not...","[b, xef, xbb, xbf, r, n, r, n, r, n, r, n, not...","[b, xef, xbb, xbf, r, n, r, n, r, n, r, n, not...","[b, xef, xbb, xbf, r, n, r, n, r, n, r, n, not..."
1,1,1,b xef xbb xbf welcome to the world of free pla...,United States,"[b, xef, xbb, xbf, welcom, world, free, plain,...","[b, xef, xbb, xbf, welcome, world, free, plain...","[b, xef, xbb, xbf, welcom, to, the, world, of,...","[b, xef, xbb, xbf, welcome, to, the, world, of..."
2,2,2,b xef xbb xbf welcome to the world of free pla...,"Kennedy, John F. (John Fitzgerald), 1917-1963","[b, xef, xbb, xbf, welcom, world, free, plain,...","[b, xef, xbb, xbf, welcome, world, free, plain...","[b, xef, xbb, xbf, welcom, to, the, world, of,...","[b, xef, xbb, xbf, welcome, to, the, world, of..."
3,3,3,b xef xbb xbf r n r n welcome to the world of ...,"Lincoln, Abraham, 1809-1865","[b, xef, xbb, xbf, r, n, r, n, welcom, world, ...","[b, xef, xbb, xbf, r, n, r, n, welcome, world,...","[b, xef, xbb, xbf, r, n, r, n, welcom, to, the...","[b, xef, xbb, xbf, r, n, r, n, welcome, to, th..."
4,4,4,b xef xbb xbfthe project gutenberg ebook of th...,United States,"[b, xef, xbb, xbfthe, project, gutenberg, eboo...","[b, xef, xbb, xbfthe, project, gutenberg, eboo...","[b, xef, xbb, xbfthe, project, gutenberg, eboo...","[b, xef, xbb, xbfthe, project, gutenberg, eboo..."


# TF-IDF

In [127]:
## tfidf vectorizer needs sentence and not token. Hence we need to combine all the tokens back to form a string

data['clean_text_stemmed'] = [' '.join(text) for text in data['clean_text_stemmed']]
data['clean_text_lemmatized'] = [' '.join(text) for text in data['clean_text_lemmatized']]
data['clean_text_lemmatized'][0]

'b xef xbb xbf r n r n r n r n note file combine project gutenberg r n file give filenumber r n duplicate file update r n year original file include r n old subdirectory access r n file list pg catalog file change r n original etexts r n r n r n r n r n welcome world free plain vanilla electronic text r n r n etexts readable human computer r n r n etexts prepare hundred volunteer donation r n r nbelow find project gutenberg etexts r file header entire file r nthe overhead response request gopher site r nkeeper eliminate header possible r n r nhowever legal financial reason request r nheader leave beginning file post r nany general user area project gutenberg run r ndonation people like r n r nif book post header r nare request send note request attach r nheader legal protection r nloss donation hope project gutenberg go r nlong post book play musical piece etc r n r n r n r n start small print public domain etexts start r nwhy small print statement know lawyer r nthey tell sue wrong r 

In [161]:
train_data=data['clean_text_lemmatized']

In [164]:
train_data=pd.DataFrame(train_data)

In [166]:
train_data.shape

(107, 1)

In [167]:
train_data[0]

0      b xef xbb xbf r n r n r n r n note file combin...
1      b xef xbb xbf welcome world free plain vanilla...
2      b xef xbb xbf welcome world free plain vanilla...
3      b xef xbb xbf r n r n welcome world free plain...
4      b xef xbb xbfthe project gutenberg ebook unite...
5      b xef xbb xbf welcome world free plain vanilla...
6      b xef xbb xbfthe project gutenberg ebook linco...
7      b xef xbb xbfthe project gutenberg ebook linco...
8      b project gutenberg ebook alice xe x x s adven...
9      b xef xbb xbfthe project gutenberg ebook look ...
10     b xef xbb xbfthe project gutenberg ebook hunti...
11     b xef xbb xbfthe project gutenberg ebook cia w...
12     b project gutenberg ebook moby dick herman mel...
13     b xef xbb xbfthe project gutenberg ebook peter...
14     b xef xbb xbfthe project gutenberg ebook book ...
15     b xef xbb xbfthe project gutenberg ebook book ...
16     b xef xbb xbfthe project gutenberg ebook feder...
17     b xef xbb xbfthe project

In [177]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.90,
                                 min_df=0.10,
                                 use_idf=True, ngram_range=(1,4))  
tfidf_matrix = tfidf_vectorizer.fit_transform(data['clean_text_lemmatized'])

print(tfidf_matrix.shape)

(107, 22965)


# we have 107 authors so n_components giving 107

In [221]:
# SVD
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=107, algorithm='randomized', n_iter=100, random_state=122)

In [222]:
tfidf_matrix.shape

(107, 22965)

In [223]:
svd_model.fit(tfidf_matrix)

TruncatedSVD(algorithm='randomized', n_components=107, n_iter=100,
             random_state=122, tol=0.0)

In [224]:
svd_matrix_train = preprocessing_model.transform(tfidf_matrix)

In [225]:
type(svd_matrix_train)

numpy.ndarray

In [226]:
train_data=pd.DataFrame(svd_matrix_train) #new data with 107 features instead of 22965

In [227]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,97,98,99,100,101,102,103,104,105,106
0,0.111502,0.407191,0.285223,-0.366838,-0.434369,0.384149,-0.059686,0.082783,-0.046326,-0.060589,...,3.130464e-17,-3.210213e-17,-1.540075e-17,-2.137911e-17,-3.005442e-17,-3.306393e-18,-8.637026e-17,-1.513987e-17,3.011033e-17,-8.080694e-19
1,0.025885,0.120821,0.215968,-0.230507,-0.268490,0.699513,-0.101948,-0.426224,0.033161,0.025279,...,3.320369e-19,5.905514e-18,-7.440337e-18,-6.017322e-18,5.353248e-18,7.941781e-18,-5.664956e-18,1.159461e-17,4.570590e-18,-2.402185e-18
2,0.057019,0.212848,0.185989,-0.233803,-0.256472,0.650348,-0.079287,-0.406633,0.026026,0.041936,...,1.153659e-18,4.838252e-18,-2.386600e-17,-9.276705e-18,-1.358641e-17,1.023555e-17,-1.419627e-18,1.386847e-17,9.774760e-18,-1.023893e-17
3,0.030670,0.132388,0.194772,-0.211989,-0.250230,0.705206,-0.101578,-0.460221,0.032678,0.033760,...,9.059864e-18,2.005774e-18,-1.394555e-17,6.125742e-18,2.900241e-18,-2.900241e-18,7.494548e-18,-9.852687e-18,1.665267e-17,-6.164706e-18
4,0.057786,0.233684,0.244992,-0.298480,-0.306923,0.171022,-0.045235,0.170663,-0.046259,-0.088660,...,-1.079798e-17,5.817422e-18,2.713470e-18,1.548376e-18,7.804303e-18,-7.373422e-18,-2.487058e-17,4.626494e-18,-1.947498e-17,-3.805296e-18


# train test split

# Add target variable

In [231]:
train_data.shape

(107, 108)

In [233]:
pd.DataFrame(data).shape

(107, 8)

In [228]:
train_data['target']=data['author']

In [234]:
train_data.tail()

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,target
102,0.962017,-0.233212,0.017188,-0.002894,-0.001477,0.001124,-0.005381,-0.004498,-0.022527,0.003123,...,1.001102e-16,-1.523473e-17,1.550441e-18,3.494699e-17,-2.228472e-16,2.819032e-17,-1.019115e-16,-9.401748e-18,-7.358953e-17,"Hope, Anthony, 1863-1933"
103,0.165171,0.491660,-0.199769,0.053720,0.049243,-0.006913,-0.144966,-0.050039,-0.184895,0.020447,...,1.683582e-15,1.542108e-15,-3.580112e-17,-1.200902e-17,5.178891e-16,3.999144e-17,-7.205324e-16,-3.379132e-16,-2.413124e-16,"Burroughs, Edgar Rice, 1875-1950"
104,0.154651,0.469268,0.017219,-0.001537,-0.021763,-0.012948,0.051017,0.113320,0.101758,0.079206,...,-7.756045e-16,4.051613e-17,2.341737e-16,2.363307e-17,-3.312084e-16,5.430451e-16,-2.541055e-16,-1.083762e-15,-3.472568e-16,"Abbott, Edwin Abbott, 1838-1926"
105,0.967256,-0.190799,0.006137,-0.007257,0.000640,0.000025,-0.013403,-0.000434,-0.003600,-0.000027,...,3.944753e-17,-5.920951e-16,5.577517e-17,1.793621e-16,3.937211e-17,4.744833e-17,-9.690731e-17,-2.340239e-16,3.779999e-17,"Dickens, Charles, 1812-1870"
106,0.155161,0.488778,0.203278,0.110699,-0.158917,-0.023562,-0.075027,0.140438,-0.001587,0.015559,...,4.051121e-16,1.068888e-16,2.573422e-16,3.578138e-16,6.115171e-16,-1.121878e-16,-3.782917e-16,-3.699501e-16,-3.171562e-16,"Douglass, Frederick, 1818-1895"


In [239]:
train_data.isnull().sum()

train_data.dropna(inplace=True)

In [240]:
train_data_new=train_data.sample(frac=0.8,random_state=200) #random state is a seed value
val_data=train_data.drop(train_data_new.index)

In [241]:
#Performing train val split on the data
X_train, y_train = train_data_new.loc[:,train_data_new.columns!='target'], train_data_new.loc[:,'target']

X_val, y_val = val_data.loc[:,val_data.columns!='target'], val_data.loc[:,'target']

# Modelling

In [243]:
from sklearn.svm import SVC

In [244]:
clf_svm= SVC()

In [245]:
clf_svm.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [246]:
train_pred=clf_svm.predict(X_train)
val_pred=clf_svm.predict(X_val)

In [249]:
from sklearn.metrics import classification_report

In [250]:
print(classification_report(y_train,train_pred))
print(classification_report(y_val,val_pred))

                                               precision    recall  f1-score   support

              Abbott, Edwin Abbott, 1838-1926       1.00      1.00      1.00         1
                     Aesop, 620? BCE-564? BCE       1.00      0.50      0.67         2
     Barrie, J. M. (James Matthew), 1860-1937       0.00      0.00      0.00         1
      Baum, L. Frank (Lyman Frank), 1856-1919       1.00      1.00      1.00         2
             Burroughs, Edgar Rice, 1875-1950       0.60      1.00      0.75         9
                                       Canada       1.00      1.00      1.00         1
                    Carroll, Lewis, 1832-1898       0.00      0.00      0.00         3
                     Cather, Willa, 1873-1947       0.00      0.00      0.00         2
                        Coombs, Norman, 1932-       1.00      1.00      1.00         1
                        De Presno, Odd, 1944-       1.00      1.00      1.00         1
                   Descartes, René, 1596-1